In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [11]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility

idx2char = ['h', 'i', 'e', 'l', 'o']
# Teach hello: hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3

y_data = [[1, 0, 2, 3, 3, 4]]    # ihello

In [3]:
num_classes = 5
input_dim = 5  # one-hot size
hidden_size = 5  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6
learning_rate = 0.1

X = tf.placeholder(
    tf.float32, [None, sequence_length, input_dim])  # X one-hot
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

In [4]:
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell,X, 
                                     initial_state=initial_state,
                                     dtype=tf.float32)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    y_data = tf.constant([[1,1,1]])

    prediction = tf.constant([[[0.2,0.7],[0.6,0.2],[0.2, 0.9]]], dtype=tf.float32)
    weights = tf.constant([[1,1,1]], dtype=tf.float32)

    sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=prediction, targets=y_data,weights=weights)

    print("Loss:" , sequence_loss.eval())

Loss: 0.5967595


#다른 경우

In [6]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    y_data = tf.constant([[1,1,1]])

    prediction1 = tf.constant([[[0.3,0.7],[0.3,0.7],[0.3, 0.7]]], dtype=tf.float32)
    prediction2 = tf.constant([[[0.1,0.9],[0.1,0.9],[0.1, 0.9]]], dtype=tf.float32)
    
    weights = tf.constant([[1,1,1]], dtype=tf.float32)

    sequence_loss1 = tf.contrib.seq2seq.sequence_loss(logits=prediction1, targets=y_data,weights=weights)
    sequence_loss2 = tf.contrib.seq2seq.sequence_loss(logits=prediction2, targets=y_data,weights=weights)

    print("Loss1:" , sequence_loss1.eval(),"Loss2:" , sequence_loss2.eval())


Loss1: 0.5130153 Loss2: 0.3711007


# cost: sequence_loss

In [7]:
outputs,_states = tf.nn.dynamic_rnn(cell,X, initial_state=initial_state, dtype=tf.float32)

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y,weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

In [15]:
prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(2000):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X:x_one_hot})
        print(i, 'loss:',l,"prediction:", result, "true Y:", y_data)

        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\tPrediction str:",''.join(result_str))

0 loss: 1.6107725 prediction: [[2 3 3 4 4 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ellooo
1 loss: 1.5258474 prediction: [[2 3 3 3 4 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ellloo
2 loss: 1.4532484 prediction: [[2 3 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: elello
3 loss: 1.3810186 prediction: [[2 3 2 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: elelll
4 loss: 1.3161987 prediction: [[2 3 2 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: elelll
5 loss: 1.2625374 prediction: [[2 3 2 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: elelll
6 loss: 1.2124074 prediction: [[2 3 2 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: elelll
7 loss: 1.1680177 prediction: [[2 3 2 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: elelll
8 loss: 1.1323599 prediction: [[2 3 2 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: elelll
9 loss: 1.1045314 prediction: [[2 3 2 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: elelll
10 loss: 1